## lesson-3

Библиотеки

In [1]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt

import time

import warnings

In [2]:
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.8.0
TensorFlow 2.0 Hello World


In [3]:
x = tf.random.uniform([3, 3])

print("есть ли доступные GPU: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("тензор на GPU #0:  "),
print(x.device.endswith('GPU:0'))

есть ли доступные GPU: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
тензор на GPU #0:  
True


In [5]:
def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 циклов: {:0.2f}ms".format(1000*result))

# исполнение CPU
print(" CPU работает:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# исполнение на GPU #0 , если доступен
if tf.config.experimental.list_physical_devices("GPU"):
  print(" какой GPU:")
  with tf.device("GPU:0"): # или GPU:1 , для 2х GPU, GPU:2 для 3х  и т.д.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

 CPU работает:
10 циклов: 117.04ms
 какой GPU:
10 циклов: 1.00ms


### 1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

In [6]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    X, y = load_boston(return_X_y=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3333, shuffle=True, random_state=42)

In [8]:
def nn(X_train, X_test, y_train, y_test, neuron = 100, layer = 2, optimizer='adam', 
                                   loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], seed_list = range(5)):
    
    metrics_train = []
    metrics_test = []
    
    for seed in seed_list:
        tf.random.set_seed(seed)
        model = Sequential(name='my_model')
        model.add(Dense(    neuron, activation='tanh', input_shape=(13,)))  
        # входной слой + поносвязный слой с активацией ReLU
        for k in range(layer - 1):
          model.add(Dense(  neuron, activation='tanh')) 
          # Скрытый слой k из neuron_numb нейронов с активацией tanh

        model.add(Dense(1, activation='relu')) 
        # выходной слой из 10 нейронов (сколько классов, столько нейронов) + активация relu
        
        model.compile(
          optimizer=optimizer, # оптимизатор
          loss=loss, # функция потерь
          metrics=metrics, # метрика
        ) 
        
        with tf.device("GPU:0"):
            # Train the model.
            hh = model.fit(    
                X_train,
                y_train,
                epochs=15,
                batch_size=32, verbose=0
            )

            # Evaluate the model.
            eval_test = model.evaluate( 
                X_test,
                y_test, verbose=0
            )
        
        # print([hh.history[key][-1] for key in hh.history][1:], eval_test[1:])
        metrics_train.append([hh.history[key][-1] for key in hh.history][1:])
        metrics_test.append(eval_test[1:])
        
    return (dict(zip(list(hh.history.keys())[1:], np.mean(metrics_train, axis=0))), 
            dict(zip(list(hh.history.keys())[1:], np.mean(metrics_test, axis=0))))

In [9]:
%%time

nn(X_train, X_test, y_train, y_test, neuron = 300, layer = 3, optimizer='adam', 
                           loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], seed_list = range(5))

Wall time: 10.5 s


({'root_mean_squared_error': 7.906673240661621},
 {'root_mean_squared_error': 7.526357269287109})

При некоторых seed модель бывает редко расходится, я пока не нашёл что нужно настроить, чтобы она не ожиданно не расходилась, поэтому я решил зафиксировать seed = 0, 1, 2, 3, 4. Чтобы дальше эксперименты были +- честные.

### 2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

Замена метрик никак не влияет на результат. Поэтому будем менять функции потерь и считать все метрики.

In [10]:
%%time

loss_list = [tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError(), 
             tf.keras.losses.MeanAbsolutePercentageError(), tf.keras.losses.MeanSquaredLogarithmicError(), 
             tf.keras.losses.CosineSimilarity(), tf.keras.losses.Huber(), tf.keras.losses.LogCosh()]

metrics_list = [tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError(), 
                tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError(),
                tf.keras.metrics.MeanSquaredLogarithmicError(), tf.keras.metrics.CosineSimilarity(),
                tf.keras.metrics.LogCoshError(), tfa.metrics.RSquare(y_shape=(1,))]

metrics_train_loss = []
metrics_test_loss = []

for loss in loss_list:
    metrics_loss = nn(X_train, X_test, y_train, y_test, neuron = 300, layer = 3, optimizer='adam', 
                      loss=loss, metrics=metrics_list, seed_list = range(5))
    print(f'{loss.name} - {metrics_loss}\n')
    metrics_train_loss.append(metrics_loss[0])
    metrics_test_loss.append(metrics_loss[1])

mean_squared_error - ({'mean_squared_error': 62.53122787475586, 'mean_absolute_error': 5.763948154449463, 'root_mean_squared_error': 7.906898498535156, 'mean_absolute_percentage_error': 27.293327713012694, 'mean_squared_logarithmic_error': 0.09532533586025238, 'cosine_similarity': 1.0, 'logcosh': 5.111983871459961, 'r_square': 0.29213030338287355}, {'mean_squared_error': 56.6690788269043, 'mean_absolute_error': 5.789841556549073, 'root_mean_squared_error': 7.526061344146728, 'mean_absolute_percentage_error': 30.509847259521486, 'mean_squared_logarithmic_error': 0.1062772586941719, 'cosine_similarity': 1.0, 'logcosh': 5.132154941558838, 'r_square': 0.24664207696914672})

mean_absolute_error - ({'mean_squared_error': 66.04024200439453, 'mean_absolute_error': 5.272044467926025, 'root_mean_squared_error': 8.123397636413575, 'mean_absolute_percentage_error': 22.830429077148438, 'mean_squared_logarithmic_error': 0.09185605943202972, 'cosine_similarity': 1.0, 'logcosh': 4.649527931213379, 'r_

Таблица для трейна:

In [11]:
result_train_loss = pd.DataFrame(metrics_train_loss, index = [['loss']*len(loss_list), list(map(lambda x: x.name, loss_list))])
result_train_loss

mean_squared_error  mean_absolute_error  \
loss mean_squared_error                       62.531228             5.763948   
     mean_absolute_error                      66.040242             5.272044   
     mean_absolute_percentage_error           75.845425             5.637410   
     mean_squared_logarithmic_error           70.593533             5.617333   
     cosine_similarity                       470.998566            19.644008   
     huber_loss                               63.822935             5.150468   
     log_cosh                                 63.632400             5.227358   

                                     root_mean_squared_error  \
loss mean_squared_error                             7.906898   
     mean_absolute_error                            8.123398   
     mean_absolute_percentage_error                 8.707270   
     mean_squared_logarithmic_error                 8.401320   
     cosine_similarity                             21.678941   
     huber_loss                                     7.983115   
     log_cosh                                       7.973587   

                                     mean_absolute_percentage_error  \
loss mean_squared_error                                   27.293328   
     mean_absolute_error                                  22.830429   
     mean_absolute_percentage_error                       22.734994   
     mean_squared_logarithmic_error                       24.355475   
     cosine_similarity                                    83.468491   
     huber_loss                                           21.916277   
     log_cosh                                             22.443189   

                                     mean_squared_logarithmic_error  \
loss mean_squared_error                                    0.095325   
     mean_absolute_error                                   0.091856   
     mean_absolute_percentage_error                        0.107252   
     mean_squared_logarithmic_error                        0.098842   
     cosine_similarity                                     2.982859   
     huber_loss                                            0.086571   
     log_cosh                                              0.087319   

                                     cosine_similarity    logcosh  r_square  
loss mean_squared_error                       1.000000   5.111984  0.292130  
     mean_absolute_error                      1.000000   4.649528  0.252407  
     mean_absolute_percentage_error           1.000000   5.012864  0.141410  
     mean_squared_logarithmic_error           1.000000   4.983778  0.200863  
     cosine_similarity                        0.987537  18.950983 -4.331826  
     huber_loss                               1.000000   4.528849  0.277508  
     log_cosh                                 1.000000   4.600849  0.279665

Таблица для теста:

In [12]:
result_test_loss = pd.DataFrame(metrics_test_loss, index = [['loss']*len(loss_list), list(map(lambda x: x.name, loss_list))])
result_test_loss

mean_squared_error  mean_absolute_error  \
loss mean_squared_error                       56.669079             5.789842   
     mean_absolute_error                      48.253551             4.651135   
     mean_absolute_percentage_error           55.433896             4.650396   
     mean_squared_logarithmic_error           56.571804             5.112572   
     cosine_similarity                       403.070752            18.183127   
     huber_loss                               48.443777             4.596914   
     log_cosh                                 48.959884             4.647763   

                                     root_mean_squared_error  \
loss mean_squared_error                             7.526061   
     mean_absolute_error                            6.940051   
     mean_absolute_percentage_error                 7.443907   
     mean_squared_logarithmic_error                 7.520535   
     cosine_similarity                             20.051278   
     huber_loss                                     6.956549   
     log_cosh                                       6.995881   

                                     mean_absolute_percentage_error  \
loss mean_squared_error                                   30.509847   
     mean_absolute_error                                  23.003847   
     mean_absolute_percentage_error                       21.360466   
     mean_squared_logarithmic_error                       25.569368   
     cosine_similarity                                    82.402313   
     huber_loss                                           22.112035   
     log_cosh                                             22.746242   

                                     mean_squared_logarithmic_error  \
loss mean_squared_error                                    0.106277   
     mean_absolute_error                                   0.079515   
     mean_absolute_percentage_error                        0.085609   
     mean_squared_logarithmic_error                        0.094213   
     cosine_similarity                                     2.789623   
     huber_loss                                            0.078399   
     log_cosh                                              0.080996   

                                     cosine_similarity    logcosh  r_square  
loss mean_squared_error                       1.000000   5.132155  0.246642  
     mean_absolute_error                      1.000000   4.030723  0.358518  
     mean_absolute_percentage_error           1.000000   4.040797  0.263063  
     mean_squared_logarithmic_error           1.000000   4.478068  0.247935  
     cosine_similarity                        0.984615  17.491107 -4.358417  
     huber_loss                               1.000000   3.974778  0.355989  
     log_cosh                                 1.000000   4.027292  0.349128

Сравним все модели на тесте с помощью метрики r2

In [13]:
sorted(zip(map(lambda x: x.name, loss_list), map(lambda x: x['r_square'], metrics_test_loss)), key=lambda i: i[1], reverse=True)

[('mean_absolute_error', 0.35851799249649047),
 ('huber_loss', 0.3559891104698181),
 ('log_cosh', 0.3491280317306519),
 ('mean_absolute_percentage_error', 0.26306259632110596),
 ('mean_squared_logarithmic_error', 0.24793524742126466),
 ('mean_squared_error', 0.24664207696914672),
 ('cosine_similarity', -4.3584168434143065)]

Видно, что модели с ошибками huber_loss, log_cosh и mean_squared_error справились лучше всех. mean_absolute_error, mean_absolute_percentage_error и mean_squared_logarithmic_error справились хуже. модель с cosine_similarity вообще разошлась.